# Data ETL

## Imports and Setup

In [ ]:
import pandas as pd
import openpyxl
from datetime import datetime
import os
import numpy as np
from statsmodels.tsa.seasonal import STL
from scipy import interpolate

# Path to your Excel file
file_path = 'data/raw/Bloomberg_Data.xlsx'

# Define which sheets use column C instead of column B
use_column_c = [
    "US_Building_Permits",
    "US _BP_Single_Housing",
    "US_Housing_Start",
    "US_New_Home_Sales",
    "US_Existing_Home _Sales",
    "US Existing_Single_Home_Sales",
    "CAD_Housing_Start"
]

# Define sheets to ignore
sheets_to_ignore = [
    "US_Population_Growth_Rate_Bloom"
]

## Function Definitions

### Core Data Processing Functions


In [ ]:
def normalize_date(date):
    """Normalize date to end of month, with special handling for quarterly data"""
    if pd.isna(date):
        return None
    
    # Convert to datetime if not already
    if not isinstance(date, datetime):
        date = pd.to_datetime(date)
    
    # Get the last day of the month
    year = date.year
    month = date.month
    
    # Create end of month date
    if month == 12:
        end_of_month = datetime(year, 12, 31)
    else:
        end_of_month = datetime(year, month + 1, 1) - pd.Timedelta(days=1)
    
    return end_of_month.date()

def normalize_quarterly_date(date):
    """Normalize quarterly date to end of quarter"""
    if pd.isna(date):
        return None
    
    # Convert to datetime if not already
    if not isinstance(date, datetime):
        date = pd.to_datetime(date)
    
    year = date.year
    month = date.month
    
    # Map to end of quarter
    if month in [1, 2, 3]:  # Q1
        return datetime(year, 3, 31).date()
    elif month in [4, 5, 6]:  # Q2
        return datetime(year, 6, 30).date()
    elif month in [7, 8, 9]:  # Q3
        return datetime(year, 9, 30).date()
    else:  # Q4
        return datetime(year, 12, 31).date()

def extract_sheet_data(file_path, sheet_name, use_col_c):
    """Extract data from a specific sheet with improved data cleaning"""
    # Determine which column to use
    data_column = 'C' if sheet_name in use_col_c else 'B'
    
    # Read the sheet starting from row 7 (index 6 in pandas)
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    
    # Extract dates from column A and values from the appropriate column
    # Row 7 in Excel is index 6 in pandas (0-indexed)
    dates = df.iloc[6:, 0]  # Column A, starting from row 7
    
    if data_column == 'C':
        values = df.iloc[6:, 2]  # Column C
    else:
        values = df.iloc[6:, 1]  # Column B
    
    # Create a temporary dataframe
    temp_df = pd.DataFrame({
        'date': dates,
        'value': values
    })
    
    # Remove rows where value is NaN or empty
    temp_df = temp_df.dropna(subset=['value'])
    
    # Remove rows where date is NaN
    temp_df = temp_df.dropna(subset=['date'])
    
    # Determine the appropriate date normalization based on sheet name
    # Population growth data needs special quarterly normalization
    if sheet_name == 'US_Population_Growth_Rate_FRED':
        temp_df['date'] = temp_df['date'].apply(normalize_quarterly_date)
    else:
        temp_df['date'] = temp_df['date'].apply(normalize_date)
    
    # Remove any rows where date normalization failed
    temp_df = temp_df.dropna(subset=['date'])
    
    # Rename value column to sheet name
    temp_df = temp_df.rename(columns={'value': sheet_name})
    
    return temp_df

def is_row_worth_keeping(row, important_columns, min_important_values=5):
    """
    Determine if a row is worth keeping based on the number of important values.
    A row is worth keeping if it has at least min_important_values non-null values
    in important columns (excluding CPI-only rows).
    """
    # Count non-null values in important columns
    non_null_count = row[important_columns].notna().sum()
    
    # Special case: if the row only has CPI data, drop it
    if non_null_count == 0 and row.get('US_CPI') is not None:
        return False
    
    # Keep rows with sufficient important data
    return non_null_count >= min_important_values

### Canada-US Softwood Lumber Exports Data Processing Functions

In [ ]:
def extract_softwood_data(file_path):
    """Extract and process CAD_Softwood_Export_to_US data with STL decomposition for missing values"""
    
    # Read the softwood export sheet
    df = pd.read_excel(file_path, sheet_name='CAD_Softwood_Export_to_US', header=None)
    
    # Extract dates and values (data starts at row 6, index 6)
    dates = df.iloc[6:, 0]  # Column A
    values = df.iloc[6:, 1]  # Column B
    
    # Create dataframe
    softwood_df = pd.DataFrame({
        'date': dates,
        'value': values
    })
    
    # Remove rows where both date and value are NaN
    softwood_df = softwood_df.dropna(subset=['date'])
    
    # Convert dates to datetime and normalize to end of month
    softwood_df['date'] = pd.to_datetime(softwood_df['date'])
    softwood_df['date'] = softwood_df['date'].apply(normalize_date)
    
    # Remove any rows where date normalization failed
    softwood_df = softwood_df.dropna(subset=['date'])
    
    # Sort by date
    softwood_df = softwood_df.sort_values('date').reset_index(drop=True)
    
    print(f"Extracted {len(softwood_df)} monthly data points")
    print(f"Date range: {softwood_df['date'].min()} to {softwood_df['date'].max()}")
    print(f"Missing values: {softwood_df['value'].isna().sum()}")
    
    return softwood_df

def impute_missing_values_stl(df):
    """Impute missing values using STL decomposition with seasonal interpolation"""
    
    # Convert values to numeric to ensure proper data type
    df_copy = df.copy()
    df_copy['value'] = pd.to_numeric(df_copy['value'], errors='coerce')
    
    # Create a complete date range for monthly data
    start_date = df_copy['date'].min()
    end_date = df_copy['date'].max()
    complete_dates = pd.date_range(start=start_date, end=end_date, freq='ME')  # Use 'ME' instead of 'M'
    complete_dates = [normalize_date(d) for d in complete_dates]
    
    # Create complete dataframe
    complete_df = pd.DataFrame({'date': complete_dates})
    complete_df = complete_df.merge(df_copy, on='date', how='left')
    
    # Check if we have enough data for STL decomposition
    non_null_count = complete_df['value'].notna().sum()
    total_count = len(complete_df)
    
    print(f"Data completeness: {non_null_count}/{total_count} ({non_null_count/total_count*100:.1f}%)")
    
    if non_null_count < 24:  # Need at least 2 years of data for STL
        print("Warning: Insufficient data for STL decomposition. Using linear interpolation instead.")
        complete_df['value'] = complete_df['value'].interpolate(method='linear')
    else:
        # Prepare data for STL decomposition
        complete_df = complete_df.set_index('date')
        
        # Store original missing mask before filling
        original_missing_mask = complete_df['value'].isna()
        
        # Forward fill and backward fill to handle edge cases for STL
        ts_filled = complete_df['value'].ffill().bfill()
        
        # Perform STL decomposition
        try:
            # Use the working parameters: seasonal=11, period=12
            stl = STL(ts_filled, seasonal=11, period=12, robust=True)
            result = stl.fit()
            
            # Use seasonal component for interpolation of missing values
            seasonal_component = result.seasonal
            trend_component = result.trend
            residual_component = result.resid
            
            # For missing values, use trend + seasonal components
            if original_missing_mask.any():
                # Fill missing values with trend + seasonal
                complete_df.loc[original_missing_mask, 'value'] = (
                    trend_component[original_missing_mask] + 
                    seasonal_component[original_missing_mask]
                )
            
            print("STL decomposition completed successfully")
            print(f"Imputed {original_missing_mask.sum()} missing values using STL")
            
            # Show some statistics
            print(f"STL Statistics - Trend range: {trend_component.min():.0f} to {trend_component.max():.0f}")
            print(f"STL Statistics - Seasonal range: {seasonal_component.min():.0f} to {seasonal_component.max():.0f}")
            
        except Exception as e:
            print(f"STL decomposition failed: {e}")
            print("Falling back to linear interpolation")
            complete_df['value'] = complete_df['value'].interpolate(method='linear')
    
    # Reset index and return
    complete_df = complete_df.reset_index()
    return complete_df

def aggregate_monthly_to_quarterly(df):
    """Aggregate monthly data to quarterly data"""
    
    # Convert date column to datetime for proper resampling
    df_copy = df.copy()
    df_copy['date'] = pd.to_datetime(df_copy['date'])
    
    # Set date as index for resampling
    df_indexed = df_copy.set_index('date')
    
    # Resample to quarterly (end of quarter) and sum the values
    quarterly_df = df_indexed.resample('QE').sum().reset_index()  # Use 'QE' instead of 'Q'
    
    # Convert quarterly dates to end of quarter format
    quarterly_df['date'] = quarterly_df['date'].apply(normalize_quarterly_date)
    
    # Rename the value column
    quarterly_df = quarterly_df.rename(columns={'value': 'CAD_Softwood_Export_to_US'})
    
    print(f"Aggregated to {len(quarterly_df)} quarterly data points")
    print(f"Quarterly date range: {quarterly_df['date'].min()} to {quarterly_df['date'].max()}")
    
    return quarterly_df

def impute_master_df_softwood_stl(master_df):
    """
    Impute missing values in CAD_Softwood_Export_to_US using STL decomposition.
    Handles edge missing values by extrapolating trend + seasonal components.
    
    Parameters:
    -----------
    master_df : pandas.DataFrame
        Master dataframe with 'Date' and 'CAD_Softwood_Export_to_US' columns
        
    Returns:
    --------
    pandas.DataFrame
        Updated dataframe with imputed softwood values
    """
    
    print("\n" + "="*60)
    print("STL Imputation for Master DataFrame Softwood Values")
    print("="*60)
    
    # Create a copy to avoid modifying the original
    df_copy = master_df.copy()
    
    # Extract Date and CAD_Softwood_Export_to_US columns
    softwood_data = df_copy[['Date', 'CAD_Softwood_Export_to_US']].copy()
    
    # Convert Date to datetime and sort
    softwood_data['Date'] = pd.to_datetime(softwood_data['Date'])
    softwood_data = softwood_data.sort_values('Date').reset_index(drop=True)
    
    # Check initial missing values
    initial_missing = softwood_data['CAD_Softwood_Export_to_US'].isna().sum()
    total_values = len(softwood_data)
    
    print(f"Initial analysis:")
    print(f"- Total data points: {total_values}")
    print(f"- Missing values: {initial_missing} ({initial_missing/total_values*100:.1f}%)")
    print(f"- Date range: {softwood_data['Date'].min().date()} to {softwood_data['Date'].max().date()}")
    
    if initial_missing == 0:
        print("No missing values found. Returning original dataframe.")
        return df_copy
    
    # Convert values to numeric
    softwood_data['CAD_Softwood_Export_to_US'] = pd.to_numeric(
        softwood_data['CAD_Softwood_Export_to_US'], errors='coerce'
    )
    
    # Check if we have enough data for STL decomposition
    non_null_count = softwood_data['CAD_Softwood_Export_to_US'].notna().sum()
    
    print(f"\nSTL Decomposition Setup:")
    print(f"- Non-null values: {non_null_count}")
    print(f"- Data completeness: {non_null_count/total_values*100:.1f}%")
    
    if non_null_count < 8:  # Need at least 2 years of quarterly data for STL
        print("Warning: Insufficient data for STL decomposition. Using linear interpolation instead.")
        softwood_data['CAD_Softwood_Export_to_US'] = softwood_data['CAD_Softwood_Export_to_US'].interpolate(method='linear')
    else:
        # Prepare data for STL decomposition
        softwood_ts = softwood_data.set_index('Date')['CAD_Softwood_Export_to_US']
        
        # Store original missing mask
        original_missing_mask = softwood_ts.isna()
        
        # Forward fill and backward fill to handle edge cases for STL
        ts_filled = softwood_ts.ffill().bfill()
        
        try:
            # Perform STL decomposition with quarterly parameters
            print("Performing STL decomposition...")
            print("- Parameters: seasonal=11, period=4 (quarterly), robust=True")
            
            stl = STL(ts_filled, seasonal=11, period=4, robust=True)
            result = stl.fit()
            
            # Extract components
            trend_component = result.trend
            seasonal_component = result.seasonal
            residual_component = result.resid
            
            print("STL decomposition completed successfully!")
            print(f"- Trend range: {trend_component.min():.0f} to {trend_component.max():.0f}")
            print(f"- Seasonal range: {seasonal_component.min():.0f} to {seasonal_component.max():.0f}")
            print(f"- Residual std: {residual_component.std():.0f}")
            
            # Impute missing values using trend + seasonal components
            if original_missing_mask.any():
                imputed_values = trend_component[original_missing_mask] + seasonal_component[original_missing_mask]
                softwood_ts.loc[original_missing_mask] = imputed_values
                
                print(f"\nImputation Results:")
                print(f"- Imputed {original_missing_mask.sum()} missing values")
                print(f"- Imputed value range: {imputed_values.min():.0f} to {imputed_values.max():.0f}")
                
                # Show some examples of imputed values
                imputed_indices = softwood_ts.index[original_missing_mask]
                print(f"- Sample imputed dates: {[d.date() for d in imputed_indices[:3]]}")
            
            # Update the dataframe
            softwood_data['CAD_Softwood_Export_to_US'] = softwood_ts.values
            
        except Exception as e:
            print(f"STL decomposition failed: {e}")
            print("Falling back to linear interpolation")
            softwood_data['CAD_Softwood_Export_to_US'] = softwood_data['CAD_Softwood_Export_to_US'].interpolate(method='linear')
    
    # Ensure Date column is in the same format as the original dataframe
    # Convert back to the original Date format (object type with date objects)
    softwood_data['Date'] = pd.to_datetime(softwood_data['Date']).dt.date
    
    # Create a mapping dictionary for imputed values to avoid merge duplicates
    imputed_mapping = dict(zip(softwood_data['Date'], softwood_data['CAD_Softwood_Export_to_US']))
    
    # Apply imputed values directly to avoid merge duplicates
    df_copy['CAD_Softwood_Export_to_US'] = df_copy['Date'].map(imputed_mapping).fillna(df_copy['CAD_Softwood_Export_to_US'])
    
    # Final verification
    final_missing = df_copy['CAD_Softwood_Export_to_US'].isna().sum()
    print(f"\nFinal Results:")
    print(f"- Missing values after imputation: {final_missing}")
    print(f"- Imputation success: {'✓' if final_missing == 0 else '✗'}")
    
    return df_copy


## Canada-US Softwood Lumber Exports Data Processing


In [ ]:
# Process CAD_Softwood_Export_to_US data
print("Processing CAD_Softwood_Export_to_US data...")
print("="*50)

# Extract monthly softwood data
softwood_monthly = extract_softwood_data(file_path)

# Impute missing values using STL decomposition
print("\nImputing missing values using STL decomposition...")
softwood_complete = impute_missing_values_stl(softwood_monthly)

# Aggregate monthly data to quarterly
print("\nAggregating monthly data to quarterly...")
softwood_quarterly = aggregate_monthly_to_quarterly(softwood_complete)

# Check for duplicate dates and remove them
print("\nChecking for duplicate dates...")
initial_count = len(softwood_quarterly)
duplicate_mask = softwood_quarterly.duplicated(subset=['date'], keep='first')
duplicate_count = duplicate_mask.sum()

if duplicate_count > 0:
    print(f"Found {duplicate_count} duplicate date(s). Removing duplicates...")
    duplicate_dates = softwood_quarterly[duplicate_mask]['date'].tolist()
    print(f"Duplicate dates: {duplicate_dates}")
    
    # Keep only the first occurrence of each date
    softwood_quarterly = softwood_quarterly[~duplicate_mask].reset_index(drop=True)
    final_count = len(softwood_quarterly)
    print(f"Removed {initial_count - final_count} duplicate row(s).")
else:
    print("No duplicate dates found.")

print(f"\nFinal softwood quarterly data: {len(softwood_quarterly)} data points")
print("Sample of processed data:")
print(softwood_quarterly.head(10))


### STL Decomposition: Mathematical Foundation and Rationale

**STL (Seasonal and Trend decomposition using Loess)** is a robust time series decomposition method that separates a time series into three components:

#### Mathematical Model
For a time series $Y_t$, STL decomposes it as:
$$Y_t = T_t + S_t + R_t$$

Where:
- **$T_t$** = Trend component (long-term movement)
- **$S_t$** = Seasonal component (recurring patterns within a year)
- **$R_t$** = Residual component (irregular fluctuations)

#### Why STL for Canada-US Softwood Lumber Exports Data?

1. **Seasonal Nature of Construction**: Softwood lumber exports exhibit strong seasonal patterns due to:
   - Construction activity peaks in spring/summer
   - Winter slowdowns in building activity
   - Weather-dependent construction cycles

2. **Robust to Outliers**: STL uses Loess (Locally Weighted Scatterplot Smoothing) which is:
   - Less sensitive to extreme values than traditional methods
   - Handles irregular patterns better than moving averages
   - Preserves local patterns while smoothing global trends

3. **Flexible Seasonal Patterns**: Unlike fixed seasonal models, STL:
   - Allows seasonal patterns to evolve over time
   - Handles changing amplitude of seasonal effects
   - Adapts to structural breaks in the data

#### Our Implementation Parameters

- **`seasonal=11`**: Uses 11-point seasonal window for monthly data
- **`period=12`**: Assumes 12-month seasonal cycle (annual pattern)
- **`robust=True`**: Uses robust statistics to handle outliers

#### Missing Value Imputation Strategy

For missing values at time $t$, we estimate:
$$\hat{Y}_t = \hat{T}_t + \hat{S}_t$$

This approach:
- Preserves the underlying seasonal structure
- Maintains trend consistency
- Provides more realistic estimates than simple interpolation
- Accounts for the specific month's typical seasonal behavior

#### Advantages Over Alternatives

- **vs. Linear Interpolation**: Captures seasonal patterns, not just linear trends
- **vs. Moving Averages**: More flexible and robust to outliers
- **vs. ARIMA**: Simpler, more interpretable, and handles missing values naturally
- **vs. Simple Seasonal Decomposition**: More robust and handles irregular patterns better


## Data Extraction

In [ ]:
# Load the Excel file to get all sheet names
excel_file = pd.ExcelFile(file_path)
sheet_names = excel_file.sheet_names

print(f"Found {len(sheet_names)} sheets in the Excel file\n")

# Extract data from all sheets
all_dataframes = []

# Add the processed softwood quarterly data first
print(f"Adding processed softwood data...", end=' ')
all_dataframes.append(softwood_quarterly)
print(f"✓ ({len(softwood_quarterly)} data points)")

for sheet_name in sheet_names:
    # Skip ignored sheets and the softwood sheet (already processed)
    if sheet_name in sheets_to_ignore or sheet_name == 'CAD_Softwood_Export_to_US':
        if sheet_name == 'CAD_Softwood_Export_to_US':
            print(f"Skipping: {sheet_name} (processed separately)")
        else:
            print(f"Skipping: {sheet_name} (ignored)")
        continue
    
    print(f"Processing: {sheet_name}...", end=' ')
    try:
        df = extract_sheet_data(file_path, sheet_name, use_column_c)
        all_dataframes.append(df)
        print(f"✓ ({len(df)} data points)")
    except Exception as e:
        print(f"✗ Error: {e}")


## Data Merging

In [ ]:
# Merge all dataframes on the date column
print("\nMerging all data into master dataframe...")

master_df = all_dataframes[0]
for df in all_dataframes[1:]:
    master_df = master_df.merge(df, on='date', how='outer')

# Sort by date (most recent first)
master_df = master_df.sort_values('date', ascending=False)

# Rename date column to 'Date'
master_df = master_df.rename(columns={'date': 'Date'})

# Reset index
master_df = master_df.reset_index(drop=True)

## Data Filtering

In [ ]:
print(f"\nBefore filtering:")
print(f"Total rows: {len(master_df)}")

# Identify important columns (all except Date and US_CPI)
important_cols = [col for col in master_df.columns if col not in ['Date', 'US_CPI']]

# Apply improved filtering logic
print(f"\nApplying improved filtering logic...")

# Create a mask for rows worth keeping
keep_mask = master_df.apply(lambda row: is_row_worth_keeping(row, important_cols, min_important_values=8), axis=1)

# Filter the dataframe
master_df_filtered = master_df[keep_mask].copy()

print(f"\nAfter improved filtering (minimum 8 important non-null values, excluding CPI-only rows):")
print(f"Total rows: {len(master_df_filtered)}")
print(f"Rows removed: {len(master_df) - len(master_df_filtered)}")

# Show some statistics about the filtering
print(f"\nFiltering statistics:")
print(f"- Rows with only CPI data: {len(master_df[(master_df[important_cols].notna().sum(axis=1) == 0) & (master_df['US_CPI'].notna())])}")
print(f"- Rows with 1-7 important values: {len(master_df[(master_df[important_cols].notna().sum(axis=1) >= 1) & (master_df[important_cols].notna().sum(axis=1) < 8)])}")
print(f"- Rows with 8+ important values: {len(master_df[master_df[important_cols].notna().sum(axis=1) >= 8])}")

## Data Quality Check

In [ ]:
print("\nData Quality Check - Detailed analysis of filtered data:")
check_cols = [col for col in master_df_filtered.columns if col != 'Date']
important_cols_check = [col for col in check_cols if col != 'US_CPI']

print(f"\nNon-null value distribution in filtered data:")
non_null_counts = master_df_filtered[check_cols].notna().sum(axis=1)
important_non_null_counts = master_df_filtered[important_cols_check].notna().sum(axis=1)

print(f"- Total non-null values per row: min={non_null_counts.min()}, max={non_null_counts.max()}, mean={non_null_counts.mean():.1f}")
print(f"- Important non-null values per row: min={important_non_null_counts.min()}, max={important_non_null_counts.max()}, mean={important_non_null_counts.mean():.1f}")

# Check for any remaining sparse rows
sparse_rows = []
for idx, row in master_df_filtered.iterrows():
    non_null = row[check_cols].notna().sum()
    important_non_null = row[important_cols_check].notna().sum()
    if important_non_null < 8:
        sparse_rows.append((row['Date'], important_non_null, non_null))

if sparse_rows:
    print(f"\n⚠️  Found {len(sparse_rows)} rows with fewer than 8 important non-null values:")
    for date, important_count, total_count in sparse_rows:
        print(f"  {date}: {important_count} important, {total_count} total non-null values")
else:
    print(f"\n✓ All rows have at least 8 important non-null values!")

# Show population growth data alignment check
print(f"\nPopulation Growth Data Check:")
pop_growth_data = master_df_filtered[['Date', 'US_Population_Growth_Rate_FRED']].dropna()
print(f"- Population growth data points: {len(pop_growth_data)}")
if len(pop_growth_data) > 0:
    print(f"- Date range: {pop_growth_data['Date'].min()} to {pop_growth_data['Date'].max()}")
    print(f"- Sample values: {pop_growth_data.head(3)['US_Population_Growth_Rate_FRED'].tolist()}")

## STL Imputation for Master DataFrame


In [ ]:
# Apply STL imputation to fill missing CAD_Softwood_Export_to_US values
print("Applying STL imputation to master dataframe...")

# Check missing values before imputation
before_missing = master_df_filtered['CAD_Softwood_Export_to_US'].isna().sum()
before_total = len(master_df_filtered)
print(f"\nBefore STL imputation:")
print(f"- Total rows: {before_total}")
print(f"- Missing CAD_Softwood_Export_to_US values: {before_missing} ({before_missing/before_total*100:.1f}%)")

# Apply STL imputation
master_df_final = impute_master_df_softwood_stl(master_df_filtered)

# Check for and remove any duplicate rows that may have been created
print("\nChecking for duplicate rows...")
initial_rows = len(master_df_final)
duplicate_mask = master_df_final.duplicated(subset=['Date'], keep='first')
duplicate_count = duplicate_mask.sum()

if duplicate_count > 0:
    print(f"Found {duplicate_count} duplicate row(s). Removing duplicates...")
    duplicate_dates = master_df_final[duplicate_mask]['Date'].tolist()
    print(f"Duplicate dates: {duplicate_dates}")
    
    # Keep only the first occurrence of each date
    master_df_final = master_df_final[~duplicate_mask].reset_index(drop=True)
    final_rows = len(master_df_final)
    print(f"Removed {initial_rows - final_rows} duplicate row(s).")
else:
    print("No duplicate rows found.")

# Check missing values after imputation
after_missing = master_df_final['CAD_Softwood_Export_to_US'].isna().sum()
after_total = len(master_df_final)
print(f"\nAfter STL imputation:")
print(f"- Total rows: {after_total}")
print(f"- Missing CAD_Softwood_Export_to_US values: {after_missing} ({after_missing/after_total*100:.1f}%)")

# Verify no missing values remain
if after_missing == 0:
    print("✓ SUCCESS: All missing CAD_Softwood_Export_to_US values have been filled!")
else:
    print(f"⚠️  WARNING: {after_missing} missing values still remain")

# Show summary statistics
print(f"\nImputation Summary:")
print(f"- Values imputed: {before_missing - after_missing}")
print(f"- Imputation success rate: {((before_missing - after_missing) / before_missing * 100) if before_missing > 0 else 100:.1f}%")

# Display sample of the final data
print(f"\nSample of final data with imputed values:")
sample_data = master_df_final[['Date', 'CAD_Softwood_Export_to_US']].head(10)
print(sample_data)


## Summary

In [ ]:
print(f"\n{'='*60}")
print(f"Master DataFrame Summary:")
print(f"{'='*60}")
print(f"Original total rows: {len(master_df)}")
print(f"Filtered total rows: {len(master_df_filtered)}")
print(f"Final total rows (after STL imputation): {len(master_df_final)}")
print(f"Total columns: {len(master_df_final.columns)} (Date + {len(master_df_final.columns)-1} variables)")
print(f"Date range: {master_df_final['Date'].min()} to {master_df_final['Date'].max()}")
print(f"\nColumns: {', '.join(master_df_final.columns.tolist())}")

# Show softwood data completeness
softwood_missing = master_df_final['CAD_Softwood_Export_to_US'].isna().sum()
print(f"\nCAD_Softwood_Export_to_US completeness: {((len(master_df_final) - softwood_missing) / len(master_df_final) * 100):.1f}% ({softwood_missing} missing values)")

In [ ]:
master_df_final.head()

In [ ]:
master_df_final.info()

In [ ]:
master_df_final.describe()

### Save Output

In [ ]:
output_dir = 'data/processed'
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# Save to CSV
output_file = os.path.join(output_dir, 'bloomberg_master_dataframe.csv')
master_df_final.to_csv(output_file, index=False)
print(f"Master dataframe saved to: {output_file}")

In [ ]:
# Save to Excel
output_excel = os.path.join(output_dir, 'bloomberg_master_dataframe.xlsx')
master_df_final.to_excel(output_excel, index=False)
print(f"Master dataframe saved to: {output_excel}")